# 🚀 EnfoadsIA - Backend Production (T4 GPU)

Este notebook está diseñado para ejecutar el ecosistema completo de **EnfoadsIA** en la nube. 

---

In [ ]:
# @title 📁 Opcional: Persistencia con Google Drive
# @markdown Activa esta celda si quieres guardar tus assets y configuraciones permanentemente.
MOUNT_DRIVE = False #@param {type:"boolean"}

if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    # Crear un enlace simbólico para la carpeta data
    !mkdir -p /content/drive/MyDrive/EnfoadsIA/data
    !ln -s /content/drive/MyDrive/EnfoadsIA/data ./data
    print("✅ Google Drive montado y carpeta 'data' vinculada.")
else:
    print("⚠️  Persistencia desactivada. Los archivos se borrarán al cerrar la sesión.")

In [ ]:
# @title 🛠️ 1. Entorno y Dependencias
import os, sys, time
REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git"

print("📡 Configurando repositorio...")
if not os.path.exists("FoadsIA"):
    !git clone $REPO_URL
else:
    !cd FoadsIA && git pull

os.chdir("FoadsIA/backend")

print("📦 Instalando dependencias...")
!apt-get update -qq && apt-get install -y -qq ffmpeg libsm6 libxext6 libgl1 > /dev/null 2>&1
!pip install -q -r requirements.txt

import torch
print(f"✅ GPU LISTA: {torch.cuda.get_device_name(0)}")

In [ ]:
# @title 🚀 2. Ejecutar Servidor (Solución Error 4018)
import os
from pyngrok import ngrok

# 1. Configurar Token
AUTH_TOKEN = "2yHQiBeYhFdbJSiK31054jtsKkw_54yvtD5Cs9mK2yhFgQ2j" #@param {type:"string"}
!ngrok config add-authtoken {AUTH_TOKEN}

# 2. Limpieza total de procesos previos
print("🧹 Limpiando procesos antiguos...")
try:
    !pkill ngrok
    ngrok.kill()
except: pass

print("\n🌍 Abriendo túnel de acceso...")
try:
    # 3. Abrimos el túnel PRIMERO para obtener la URL
    public_url = ngrok.connect(5000).public_url
    print(f"\n{'='*60}")
    print(f" 📡 URL PARA EL FRONTEND: {public_url}")
    print(f"{'='*60}\n")
    
    # 4. Ejecutamos el archivo app.py (que ya no tiene ngrok interno)
    print("🚀 Iniciando Backend...")
    !python app.py
    
except Exception as e:
    print(f"❌ Error: {e}")